In [7]:
import numpy as np
import matplotlib.pyplot as plt
import xsensdeviceapi as xda
import sys
from threading import Lock
import re
import time
import threading as th
import os
import pickle
from MTwFunctions import pickle2txt

In [4]:
MTw_pickle = os.path.dirname(os.path.realpath("__file__")) + "\\MTw Pickle"
MTw_txt = os.path.dirname(os.path.realpath("__file__")) + "\\MTw data"


c:\Users\rena_\Google Drive\STG-01\Projet IMU pour estimation de pose\IMU python\MTw Pickle\test.txt


In [17]:
class XdaCallback(xda.XsCallback):
    def __init__(self, max_buffer_size = 5):
        xda.XsCallback.__init__(self)
        self.m_maxNumberOfPacketsInBuffer = max_buffer_size
        self.m_packetBuffer = list()
        self.m_lock = Lock()

    def packetAvailable(self):
        self.m_lock.acquire()
        res = len(self.m_packetBuffer) > 0
        self.m_lock.release()
        return res

    def getNextPacket(self):
        self.m_lock.acquire()
        assert(len(self.m_packetBuffer) > 0)
        oldest_packet = xda.XsDataPacket(self.m_packetBuffer.pop(0))
        self.m_lock.release()
        return oldest_packet

    def onLiveDataAvailable(self, dev, packet):
        self.m_lock.acquire()
        assert(packet is not 0)
        while len(self.m_packetBuffer) >= self.m_maxNumberOfPacketsInBuffer:
            self.m_packetBuffer.pop()
        self.m_packetBuffer.append(xda.XsDataPacket(packet))
        self.m_lock.release()

    def writeData(self, filename):
        self.m_lock.acquire()
        assert(len(self.m_packetBuffer) > 0)
        oldest_packet = xda.XsDataPacket(self.m_packetBuffer.pop(0))
        with open(filename, "ab") as file_handle:
                pickle.dump((oldest_packet.packetCounter(), oldest_packet.calibratedAcceleration(), oldest_packet.orientationMatrix(), oldest_packet.timeOfArrival().toXsString().__str__()), (file_handle))
        self.m_lock.release()
    


def stopAll():
    global device
    global control
    global Ports
    print("Arrêt de l'enregistrement, mise du dispositif en mode configuration.")
    device.stopRecording()
    device.gotoConfig()
    device.disableRadio()
    print("Fermeture du fichier mtb")
    device.closeLogFile()
    print("Fermeture du port")
    control.closePort(Ports)
    control.close()
    xda.XsControl.destruct(control)

def checkConnectedSensors(devIdAll):
    global children
    global control
    global device
    
    childUsed = np.full(np.shape(children),False).tolist()
    if children.empty():
        print("Aucun dispositif trouvé.")
        stopAll()
        raise RuntimeError("Échec. Aucun capteur MTw trouvé.")
    else:
        for i in range(len(children)):
            if children[i].connectivityState() == xda.XCS_Wireless:
                childUsed[i] = True
        
        print("Dispositifs rejetés:")
        rejects = np.array(devIdAll)[[not elem for elem in childUsed]].tolist()
        for i in range(len(rejects)):
            index = devIdAll.index(rejects[i])
            print("%d - "%index +"%s" %rejects[i])
        
        print("Dispositifs acceptés:")
        accepted = np.array(devIdAll)[childUsed].tolist()
        for i in range(len(accepted)):
            index = devIdAll.index(accepted[i])
            print("%d - "%index +"%s" %accepted[i])
        
        option = str(input('Keep current status?'+' (y/n): ')).lower().strip()
        change = []
        if option[0] == 'n':
            op2 = input("\n Type the numbers of the sensors (csv list, e.g. 1,2,3) from which status should be changed \n (if accepted than reject or the other way around):\n")
            change = [int(i) for i in re.split(",",op2)]
            for i in range(len(change)):
                if a[change[i]]:
                    device.rejectConnection(children[change[i]])
                    childUsed[change[i]] = False
                else:
                    device.acceptConnection(children[change[i]])
                    childUsed[change[i]] = True
        if sum(childUsed) == 0:
            stopAll()
            raise RuntimeError("Échec. Aucun capteur MTw trouvé.")
    devicesUsed = np.array(children)[childUsed].tolist()
    devIdUsed = np.array(devIdAll)[childUsed].tolist()
    nDevs = sum(childUsed)

    return devicesUsed, devIdUsed, nDevs

## Thread to stop recording loop ##
def key_capture_thread():
    global keep_going
    input("Press enter to stop recording.")
    keep_going = False
###################################

In [11]:
def main():
    global children
    global control
    global device
    global device
    global control
    global Ports
    global keep_going
    try:
        # Extraire la version du Xs Device Api employé
        xdaVersion = xda.XsVersion()
        xda.xdaVersion(xdaVersion)
        print("Utilisation de la version XDA: %s \n" % xdaVersion.toXsString())

        print("Creation d'un objet XsControl...")
        control = xda.XsControl_construct()
        assert(control is not 0)

        print("Détection des ports...\n")
        ports_Scan = xda.XsScanner_scanPorts(0,100,True,True)

        # Détection des dispositifs MTw ou Station Awinda
        Ports = xda.XsPortInfo()
        for i in range(ports_Scan.size()):
            if ports_Scan[i].deviceId().isWirelessMaster() or ports_Scan[i].deviceId().isAwindaXStation():
                
                Ports = ports_Scan[i]
                break

        if Ports.empty():
            raise RuntimeError("Échec. Aucun dispositif a été detecté.")

        devId = Ports.deviceId()
        print(" Dispositif trouvé avec les caractéristiques suivantes:")
        print(" Device ID: %s" % devId.toXsString())
        print(" Nom du port: %s" % Ports.portName())
        print(" Port baudrate: %s" % Ports.baudrate())

        print("Ouverture du port...")
        if not control.openPort(Ports.portName(), Ports.baudrate()):
            raise RuntimeError("Échec de l'ouverture du port.")

        # Construction d'un object pour les dispositif
        device = control.device(devId)
        firmware_version = device.firmwareVersion()

        assert(device is not 0)
        print("Dispositif: %s, avec ID: %s ouvert. \n" %(device.productCode(), device.deviceId().toXsString()))

        
        # Mettre le dispositif en mode configuration
        print("Dispositif en mode configuration...\n")
        if not device.gotoConfig():
            raise RuntimeError("Échec. Le dispositif n'a as pu être mis en mode configuration.")

        device.setOptions(xda.XSO_Orientation + xda.XSO_Calibrate + xda.XSO_RetainLiveData, 0)

        print("Creating a log file...")
        logFileName = "logfile.mtb"
        if device.createLogFile(logFileName) != xda.XRV_OK:
            raise RuntimeError("Failed to create a log file. Aborting.")
        else:
            print("Created a log file: %s" % logFileName)


        # Acquisition des fréquences pour la mise à jour des données
        fq_support = device.supportedUpdateRates(xda.XDI_None)
        index_fq = []

        while index_fq == []:
            print("Les fréquences disponibles sont:")
            print('\n'.join('{}: {}'.format(*k) for k in enumerate(fq_support)))
            choix_fq = input("Veuillez choisir une fréquence d'acquisition: ")
            if choix_fq == []:
                continue
            index_fq = fq_support.index(int(choix_fq))
        print("Fréquence d'acquisition choisi: ", fq_support[index_fq], "\n")
        updateRate = fq_support[index_fq]
        # Mise à jour de la fréquence d'acquisition
        if not device.setUpdateRate(updateRate):
            raise RuntimeError("Échec de la mise à jour du update rate.")

        
        # Acquisition du canal radio
        canaux_radio = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
        index_radio = []

        while index_radio == []:
            print("Les canaux disponibles sont: ")
            print('\n'.join('{}: {}'.format(*k) for k in enumerate(canaux_radio)))
            choix_radio = input("Veuillez choisir un canal de radio: ")
            if choix_radio == []:
                continue
            index_radio = canaux_radio.index(int(choix_radio))
        print("Canal de radio choisi: ", canaux_radio[index_radio], "\n")

        # Mise à jour du canal radio
        try:
            device.enableRadio(canaux_radio[index_radio])
        except ValueError:
            print("La radio est encore activée, veuillez débrancher l'appareil de l'ordinateur et reessayer.")


        input('\n Undock the MTw devices from the Awinda station and wait until the devices are connected (synced leds), then press enter... \n')

        # Vérifier les dispositifs détectées (nombre de capteurs MTw)
        children = device.children()

        devIdAll = []
        mtwCallbacks = []
        filenamesPCKL = []

        for i in range(len(children)):
            devIdAll.append(children[i].deviceId())
            mtwCallbacks.append(XdaCallback())
            children[i].addCallbackHandler(mtwCallbacks[i]) 

        devicesUsed, devIdUsed,nDevs = checkConnectedSensors(devIdAll)
        MTw_pickle = os.path.dirname(os.path.realpath("__file__")) + "\\MTw Pickle"
        for n in range(nDevs):
            file_PCKL = "PCKL_" + str(devId.toXsString()) + "_" + str(devIdUsed[n]) + ".txt"
            filenamesPCKL.append(os.path.join(MTw_pickle, file_PCKL))
            open(filenamesPCKL[-1], 'wb')
        ##################

        # Mis les capteurs MTw en mode acquisition des données
        print("Putting devices into measurement mode...\n")
        
        if not device.gotoMeasurement():
            raise RuntimeError("Could not put device into measurement mode. Aborting.")
    
            
        
        print("Wait 10 seconds before starting data acquisition.\n")
        time.sleep(10)


        input("Press enter to start recording...\n")
        
        print("Starting recording...")
        if not children[0].startRecording():
            raise RuntimeError("Failed to start recording. Aborting.")


        startTime = xda.XsTimeStamp_nowMs()
        ##### Test zone
        device_syncRole = device.syncRole()
        device_syncSet = device.syncSettings()

        keep_going = True
        th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()
        
        while keep_going:
            for i in range(len(mtwCallbacks)):
                callback = mtwCallbacks[i]
                filename = filenamesPCKL[i]
                if callback.packetAvailable():
                    callback.writeData(filename)
        
                    

        startEnd = xda.XsTimeStamp_nowMs() - startTime
        print("Time end (s): ", startEnd/1000, "\n")

        pickle2txt(devId, devIdAll, devIdUsed, nDevs, firmware_version, filenamesPCKL, updateRate)



        ##################

        print("Abort flushing...\n")
        if not device.abortFlushing():
            raise RuntimeError("Failed to abort flushing operation.")

        print("\nStopping recording...")
        if not device.stopRecording():
            raise RuntimeError("Failed to stop recording. Aborting.")

        print("Closing log file...")
        if not device.closeLogFile():
            raise RuntimeError("Failed to close log file. Aborting.")
            

        exit = int(input('\n Appuyez sur la touche 0 pour quitter le mode acquisition... \n'))
        if exit == 0:
            stopAll()
        
    except RuntimeError as error:
        print(error)
        sys.exit(1)

    except:
        print("An unknown fatal error has occured. Aborting.")
        sys.exit(1)
        
    else:
        print("Successful exit.")

In [18]:
main()

Utilisation de la version XDA: 2021.0.0 build 5908 rev 109668 built on 2021-04-19 

Creation d'un objet XsControl...
Détection des ports...

 Dispositif trouvé avec les caractéristiques suivantes:
 Device ID: 01200452
 Nom du port: COM12
 Port baudrate: 2000000
Ouverture du port...
Dispositif: AW-A2, avec ID: 01200452 ouvert. 

Dispositif en mode configuration...

Creating a log file...
Created a log file: logfile.mtb
Les fréquences disponibles sont:
0: 120
1: 100
2: 80
3: 60
4: 40
Fréquence d'acquisition choisi:  120 

Les canaux disponibles sont: 
0: 11
1: 12
2: 13
3: 14
4: 15
5: 16
6: 17
7: 18
8: 19
9: 20
10: 21
11: 22
12: 23
13: 24
14: 25
Canal de radio choisi:  21 

Dispositifs rejetés:
Dispositifs acceptés:
0 - 00B4876D
Putting devices into measurement mode...

Wait 10 seconds before starting data acquisition.

Starting recording...
Time end (s):  9.558 

00B4876D number of data packets:  1079
Abort flushing...


Stopping recording...
Closing log file...
Arrêt de l'enregistrement

In [10]:
print("Time end (s): ", startEnd/1000, "\n")

Time end (s):  1.737 



In [ ]:
def export_txt(devId, devIdUsed, nDevs, firmware_version):
    global matrix_nDev, acc_nDev, counter_nDev
    f_path = os.path.dirname(os.path.realpath("__file__"))
    for n in range(nDevs):
        filename = "MT_" + str(devId.toXsString()) + "_" + str(devIdUsed[n]) + ".txt"
        path = os.path.join(f_path, filename)
        file_txt = open(path, "w")
        file_txt.write("// Start Time: Unknown " + "\n")
        file_txt.write("// Update Rate: " + str(float(fq_support[index_fq])) + "Hz \n")
        file_txt.write("// Filter Profile: human (46.1) \n")
        file_txt.write("// Firmware Version: " + str(firmware_version.toXsString()) + "\n")
        file_txt.write(
            "PacketCounter\tSampleTimeFine\tYear\tMonth\tDay\tSecond\tUTC_Nano\tUTC_Year\tUTC_Month\tUTC_Day UTC_Hour\tUTC_Minute\tUTC_Second\tUTC_Valid\tAcc_X\tAcc_Y\tAcc_Z\tMat[1][1]\tMat[2][1]\tMat[3][1]\tMat[1][2]\tMat[2][2]\tMat[3][2]\tMat[1][3]\tMat[2][3]\tMat[3][3] \n")

        for i in range(len(counter_nDev[n])):
            file_txt.write(str(counter_nDev[n][i]) + "\t\t\t\t\t\t\t\t\t\t\t\t\t\t")
            matrix = matrix_nDev[n][i].reshape(9, )
            for k in range(3):
                file_txt.write('{:.6f}'.format(round(acc_nDev[n][i][k], 6)) + "\t")
            for j in range(9):
                file_txt.write('{:.6f}'.format(round(matrix[j], 6)) + "\t")
            file_txt.write("\n")
        file_txt.close()

In [48]:
f_path = os.path.dirname(os.path.realpath("__file__"))
n = 0
filename = "MT_" + str(devId.toXsString()) + "_" + str(devIdUsed[n]) + ".txt"
path = os.path.join(f_path, filename)

print(path)


c:\Users\rena_\Google Drive\STG-01\Projet IMU pour estimation de pose\IMU python\MT_01200452_6F632E79.txt


In [51]:
devIdUsed = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18"]


f_path = os.path.dirname(os.path.realpath("__file__"))
for n in range(nDevs):
    objects = []
    filename = "MT_" + str(devId.toXsString()) + "_" + str(devIdUsed[n]) + ".txt"
    with (open(filenames[n], "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    counter = [item[0] for item in objects]
    acc = [item[1] for item in objects]
    matrix = [item[2] for item in objects]
    path = os.path.join(f_path, filename)
    #path = f_path + filename
    file_txt = open(path, "w")
    file_txt.write("// Start Time: Unknown: \n")
    file_txt.write("// Update Rate: " + str(float(fq_support[index_fq])) + "Hz \n")
    file_txt.write("// Filter Profile: human (46.1) \n")
    file_txt.write("// Firmware Version: " + "\n")
    file_txt.write(
        "PacketCounter\tSampleTimeFine\tYear\tMonth\tDay\tSecond\tUTC_Nano\tUTC_Year\tUTC_Month\tUTC_Day UTC_Hour\tUTC_Minute\tUTC_Second\tUTC_Valid\tAcc_X\tAcc_Y\tAcc_Z\tMat[1][1]\tMat[2][1]\tMat[3][1]\tMat[1][2]\tMat[2][2]\tMat[3][2]\tMat[1][3]\tMat[2][3]\tMat[3][3] \n")

    for i in range(len(counter)):
        file_txt.write(str(counter[i]) + "\t\t\t\t\t\t\t\t\t\t\t\t\t\t")
        mat = matrix[i].reshape(9, )
        for k in range(3):
            file_txt.write('{:.6f}'.format(round(acc[i][k], 6)) + "\t")
        for j in range(9):
            file_txt.write('{:.6f}'.format(round(mat[j], 6)) + "\t")
        file_txt.write("\n")
    file_txt.close()
    

In [24]:
print(filenames[0])

objects = []
with (open(filenames[0],"rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
counter = [item[0] for item in objects]
acc = [item[1] for item in objects]
matrix = [item[2] for item in objects]

print(len(objects))
print("Time end (s): ", startEnd/1000, "\n")
print(matrix)

PCKL_01200452_00B4875B.txt
921
Time end (s):  15.326 

[array([[ 9.75812098e-01,  2.18608977e-01, -9.29446704e-04],
       [-2.18590937e-01,  9.75771467e-01,  9.38330907e-03],
       [ 2.95820317e-03, -8.95317789e-03,  9.99955544e-01]]), array([[ 9.75825467e-01,  2.18549419e-01, -8.99665766e-04],
       [-2.18531667e-01,  9.75784807e-01,  9.37663858e-03],
       [ 2.92713910e-03, -8.95335726e-03,  9.99955634e-01]]), array([[ 9.75836658e-01,  2.18499626e-01, -8.54756961e-04],
       [-2.18482342e-01,  9.75796061e-01,  9.35483040e-03],
       [ 2.87809542e-03, -8.94203713e-03,  9.99955877e-01]]), array([[ 9.75856711e-01,  2.18410263e-01, -7.97597794e-04],
       [-2.18393574e-01,  9.75816205e-01,  9.32637260e-03],
       [ 2.81528434e-03, -8.92701306e-03,  9.99956190e-01]]), array([[ 9.75948868e-01,  2.17999228e-01,  3.78468717e-04],
       [-2.17993243e-01,  9.75905921e-01,  9.30476082e-03],
       [ 1.65908081e-03, -9.16347441e-03,  9.99956638e-01]]), array([[ 9.75949089e-01,  2.179982

In [8]:
print(filenames[1])

objects = []
with (open(filenames[0],"rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

print(len(objects))

PCKL_01200452_00B4875C.txt
921


In [6]:
print(filenames[2])

objects = []
with (open(filenames[0],"rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

print(len(objects))

PCKL_01200452_00B4875D.txt
921


In [7]:
print(filenames[17])

objects = []
with (open(filenames[0],"rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

print(len(objects))

PCKL_01200452_00B487B7.txt
921


In [12]:
import numpy as np
import xsensdeviceapi as xda
import time
import threading as th
import sys
# Local modules
from MTwFunctions import stopAll, checkConnectedSensors, pickle2txt


In [13]:
class MTwCallback(xda.XsCallback):
    def __init__(self, max_buffer_size=1024):
        xda.XsCallback.__init__(self)
        self.m_maxNumberOfPacketsInBuffer = max_buffer_size
        self.m_packetBuffer = list()
        self.m_lock = Lock()

    def packetAvailable(self):
        self.m_lock.acquire()
        res = len(self.m_packetBuffer) > 0
        self.m_lock.release()
        return res

    def getNextPacket(self):
        self.m_lock.acquire()
        assert (len(self.m_packetBuffer) > 0)
        oldest_packet = xda.XsDataPacket(self.m_packetBuffer.pop(0))
        self.m_lock.release()
        return oldest_packet

    def onLiveDataAvailable(self, dev, packet):
        self.m_lock.acquire()
        assert (packet is not 0)
        while len(self.m_packetBuffer) >= self.m_maxNumberOfPacketsInBuffer:
            self.m_packetBuffer.pop()
        self.m_packetBuffer.append(xda.XsDataPacket(packet))
        self.m_lock.release()

    def writeData(self, filename):
        self.m_lock.acquire()
        assert (len(self.m_packetBuffer) > 0)
        oldest_packet = xda.XsDataPacket(self.m_packetBuffer.pop(0))
        with open(filename, "ab") as file_handle:
            pickle.dump((oldest_packet.packetCounter(), oldest_packet.calibratedAcceleration(),
                         oldest_packet.orientationMatrix()), (file_handle))
        self.m_lock.release()

def key_capture_thread():
        global keep_going
        input("Press enter to stop recording.")
        keep_going = False

In [14]:
## Thread to stop recording loop ##

def main():
    global keep_going
    try:
        # Extraire la version du Xs Device API employé

        print("Creation d'un objet XsControl...")
        controlDev = xda.XsControl_construct()
        assert (controlDev is not 0)

        xdaVersion = xda.XsVersion()
        xda.xdaVersion(xdaVersion)
        print("Utilisation de la version XDA: %s \n" % xdaVersion.toXsString())

        print("Détection des ports...\n")
        ports_Scan = xda.XsScanner_scanPorts(0, 100, True, True)

        # Détection d'une Station Awinda
        Ports = xda.XsPortInfo()
        for i in range(ports_Scan.size()):
            if ports_Scan[i].deviceId().isWirelessMaster() or ports_Scan[i].deviceId().isAwindaXStation():
                Ports = ports_Scan[i]
                break

        if Ports.empty():
            raise RuntimeError("Échec. Aucun dispositif a été detecté.")

        devId = Ports.deviceId()
        print(" Dispositif trouvé avec les caractéristiques suivantes: ")
        print(" Device ID: %s" % devId.toXsString())
        print(" Nom du port: %s" % Ports.portName())
        print(" Port baudrate: %s" % Ports.baudrate())

        print("Ouverture du port...")
        if not controlDev.openPort(Ports.portName(), Ports.baudrate()):
            raise RuntimeError("Échec de l'ouverture du port.")

        awinda = controlDev.device(devId)
        firmware_version = awinda.firmwareVersion()

        assert (awinda is not 0)
        print("Dispositif: %s, avec ID: %s ouvert. \n" % (awinda.productCode(), awinda.deviceId().toXsString()))

        # Mettre le dispositif en mode configuration
        print("Dispositif en mode configuration...\n")
        if not awinda.gotoConfig():
            raise RuntimeError("Échec. Le dispositif n'a as pu être mis en mode configuration.")

        awinda.setOptions(xda.XSO_Orientation + xda.XSO_Calibrate + xda.XSO_RetainLiveData, 0)

        print("Creating a log file...")
        logFileName = "logfile.mtb"
        if awinda.createLogFile(logFileName) != xda.XRV_OK:
            raise RuntimeError("Failed to create a log file. Aborting.")
        else:
            print("Created a log file: %s" % logFileName)

        # Acquisition des fréquences pour la mise à jour des données
        supportedUpdateRates = awinda.supportedUpdateRates(xda.XDI_None)
        indexRate = []

        while indexRate == []:
            print("Les fréquences disponibles sont:")
            print('\n'.join('{}: {}'.format(*k) for k in enumerate(supportedUpdateRates)))
            choix_fq = input("Veuillez choisir une fréquence d'acquisition: ")
            if choix_fq == []:
                continue
            indexRate = supportedUpdateRates.index(int(choix_fq))
        updateRate = supportedUpdateRates[indexRate]
        print("Fréquence d'acquisition choisi: ", updateRate, "\n")

        # Mise à jour de la fréquence d'acquisition
        if not awinda.setUpdateRate(updateRate):
            raise RuntimeError("Échec de la mise à jour du update rate.")

        # Acquisition du canal radio
        radioChannels = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
        indexRadio = []

        while indexRadio == []:
            print("Les canaux disponibles sont: ")
            print('\n'.join('{}: {}'.format(*k) for k in enumerate(radioChannels)))
            choix_radio = input("Veuillez choisir un canal de radio: ")
            if choix_radio == []:
                continue
            indexRadio = radioChannels.index(int(choix_radio))
        print("Canal de radio choisi: ", radioChannels[indexRadio], "\n")
        radioChannel = radioChannels[indexRadio]
        # Mise à jour du canal radio
        try:
            awinda.enableRadio(radioChannel)
        except ValueError:
            print("La radio est encore activée, veuillez débrancher l'appareil de l'ordinateur et reessayer.")

        input(
            '\n Undock the MTw devices from the Awinda station and wait until the devices are connected (synced leds), then press enter... \n')
        # Vérifier les dispositifs détectées (nombre de capteurs MTw)

        MTws = awinda.children()

        devIdAll = []
        mtwCallbacks = []
        filenamesPCKL = []
        for i in range(len(MTws)):
            devIdAll.append(MTws[i].deviceId())
            mtwCallbacks.append(MTwCallback())
            MTws[i].addCallbackHandler(mtwCallbacks[i])

        devicesUsed, devIdUsed, nDevs = checkConnectedSensors(devIdAll, MTws, controlDev, awinda, Ports)

        for n in range(nDevs):
            filenamesPCKL.append("PCKL_" + str(devId.toXsString()) + "_" + str(devIdUsed[n]) + ".txt")
            open(filenamesPCKL[-1], 'wb')
        ##################

        # Mis les capteurs MTw en mode acquisition des données
        print("Putting devices into measurement mode...\n")

        if not awinda.gotoMeasurement():
            raise RuntimeError("Could not put device into measurement mode. Aborting.")
#

        print("Wait 10 seconds before starting data acquisition.\n")
        time.sleep(10)

        input("Press enter to start recording...\n")
        print("Starting recording...")

        if not MTws[0].startRecording():
            raise RuntimeError("Failed to start recording. Aborting.")

        startTime = xda.XsTimeStamp_nowMs()
        keep_going = True
        th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()

        while keep_going:
            for i in range(len(mtwCallbacks)):
                callback = mtwCallbacks[i]
                filenamePCKL = filenamesPCKL[i]

                if callback.packetAvailable():
                    callback.writeData(filenamePCKL)

        startEnd = xda.XsTimeStamp_nowMs() - startTime
        print("Time end (s): ", startEnd / 1000, "\n")
        #####

        pickle2txt(devId, devIdAll, devIdUsed, nDevs, firmware_version, filenamesPCKL, updateRate)

        print("Abort flushing...\n")
        if not awinda.abortFlushing():
            raise RuntimeError("Failed to abort flushing operation.")

        print("\nStopping recording...")
        if not awinda.stopRecording():
            raise RuntimeError("Failed to stop recording. Aborting.")

        print("Closing log file...")
        if not awinda.closeLogFile():
            raise RuntimeError("Failed to close log file. Aborting.")

        exit = int(input('\n Appuyez sur la touche 0 pour quitter le mode acquisition... \n'))
        if exit == 0:
            stopAll(awinda, controlDev, Ports)


    except RuntimeError as error:
        print(error)
        sys.exit(1)

    except:
        print("An unknown fatal error has occured. Aborting.")
        sys.exit(1)

    else:
        print("Successful exit.")


In [15]:
main()

Creation d'un objet XsControl...
Utilisation de la version XDA: 2021.0.0 build 5908 rev 109668 built on 2021-04-19 

Détection des ports...

 Dispositif trouvé avec les caractéristiques suivantes: 
 Device ID: 01200452
 Nom du port: COM12
 Port baudrate: 2000000
Ouverture du port...
Dispositif: AW-A2, avec ID: 01200452 ouvert. 

Dispositif en mode configuration...

Creating a log file...
Created a log file: logfile.mtb
Les fréquences disponibles sont:
0: 120
1: 100
2: 80
3: 60
4: 40
Fréquence d'acquisition choisi:  120 

Les canaux disponibles sont: 
0: 11
1: 12
2: 13
3: 14
4: 15
5: 16
6: 17
7: 18
8: 19
9: 20
10: 21
11: 22
12: 23
13: 24
14: 25
Canal de radio choisi:  22 

Dispositifs rejetés:
Dispositifs acceptés:
0 - 00B4875D
Putting devices into measurement mode...

Wait 10 seconds before starting data acquisition.

Starting recording...
Time end (s):  3.739 

00B4875D number of data packets:  1460
Abort flushing...


Stopping recording...
Closing log file...
Arrêt de l'enregistremen